In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14653,2022-01-21T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1417,170,1587,200852,...,DECEDUTI: N. 5 IL 20/01/22 - N. 14 IL 19/01/22...,14.0,NaN,NaN,452404.0,106409.0,4114428.0,5265014.0,ITG,ITG1
14654,2022-01-21T17:00:00,ITA,9,Toscana,43.769231,11.255889,1340,125,1465,178923,...,NaN,10.0,NaN,NaN,465775.0,175378.0,5882398.0,4591272.0,ITI,ITI1
14655,2022-01-21T17:00:00,ITA,10,Umbria,43.106758,12.388247,196,12,208,25391,...,Si fa presente che 10 dei ricoveri non UTI app...,1.0,NaN,NaN,101337.0,36500.0,1431446.0,1839187.0,ITI,ITI2
14656,2022-01-21T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,76,6,82,5903,...,NaN,0.0,NaN,NaN,14502.0,11651.0,130984.0,281995.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14653,2022-01-21T17:00:00,19,Sicilia,1417,170,1587,200852,202439,5422,7418,...,558813,9379442,3834966.0,14.0,452404.0,106409.0,4114428.0,5265014.0,ITG,ITG1
14654,2022-01-21T17:00:00,9,Toscana,1340,125,1465,178923,180388,397,13029,...,641153,10473670,4019375.0,10.0,465775.0,175378.0,5882398.0,4591272.0,ITI,ITI1
14655,2022-01-21T17:00:00,10,Umbria,196,12,208,25391,25599,-350,2211,...,137837,3270633,606568.0,1.0,101337.0,36500.0,1431446.0,1839187.0,ITI,ITI2
14656,2022-01-21T17:00:00,2,Valle d'Aosta,76,6,82,5903,5985,-76,444,...,26153,412979,121396.0,0.0,14502.0,11651.0,130984.0,281995.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-01-21', '2022-01-20')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-21T17:00:00,1,Piemonte,2053,142,2195,170290,172485,114,14675,...,767303,13045716,3432235.0,9.0,434248.0,333055.0,4407438.0,8638278.0,ITC,ITC1
1,2022-01-21T17:00:00,2,Valle d'Aosta,76,6,82,5903,5985,-76,444,...,26153,412979,121396.0,0.0,14502.0,11651.0,130984.0,281995.0,ITC,ITC2
2,2022-01-21T17:00:00,3,Lombardia,3503,276,3779,503173,506952,-11855,32677,...,1926236,28850371,7404100.0,17.0,1203896.0,722340.0,14081100.0,14769271.0,ITC,ITC4
3,2022-01-21T17:00:00,5,Veneto,1516,181,1697,277560,279257,5362,19117,...,975213,22147161,4353395.0,21.0,666702.0,308511.0,8805864.0,13341297.0,ITH,ITH3
4,2022-01-21T17:00:00,6,Friuli Venezia Giulia,426,40,466,57762,58228,1926,4712,...,230078,5036074,1023818.0,4.0,161119.0,68959.0,2827093.0,2208981.0,ITH,ITH4
5,2022-01-21T17:00:00,7,Liguria,740,42,782,35708,36490,1153,6197,...,251540,4058742,1139236.0,4.0,188170.0,63370.0,2134900.0,1923842.0,ITC,ITC3
6,2022-01-21T17:00:00,8,Emilia-Romagna,2459,145,2604,345543,348147,-293,20654,...,895961,12945202,2527253.0,13.0,703260.0,192701.0,7878822.0,5066380.0,ITH,ITH5
7,2022-01-21T17:00:00,9,Toscana,1340,125,1465,178923,180388,397,13029,...,641153,10473670,4019375.0,10.0,465775.0,175378.0,5882398.0,4591272.0,ITI,ITI1
8,2022-01-21T17:00:00,10,Umbria,196,12,208,25391,25599,-350,2211,...,137837,3270633,606568.0,1.0,101337.0,36500.0,1431446.0,1839187.0,ITI,ITI2
9,2022-01-21T17:00:00,11,Marche,298,51,349,17734,18083,-162,5853,...,210690,2332968,1433490.0,3.0,176694.0,33996.0,1781318.0,551650.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-20T17:00:00,1,Piemonte,2056,143,2199,170172,172371,1802,15328,...,752628,12943185,3415120.0,18.0,432160.0,320468.0,4393751.0,8549434.0,ITC,ITC1
1,2022-01-20T17:00:00,2,Valle d'Aosta,78,6,84,5977,6061,190,540,...,25709,410373,121054.0,2.0,14518.0,11191.0,130856.0,279517.0,ITC,ITC2
2,2022-01-20T17:00:00,3,Lombardia,3719,271,3990,514817,518807,19414,33676,...,1893559,28648073,7378045.0,14.0,1192356.0,701203.0,14034284.0,14613789.0,ITC,ITC4
3,2022-01-20T17:00:00,5,Veneto,1547,180,1727,272168,273895,8823,21833,...,956096,22004185,4343117.0,25.0,653562.0,302534.0,8781177.0,13223008.0,ITH,ITH3
4,2022-01-20T17:00:00,6,Friuli Venezia Giulia,429,40,469,55833,56302,2399,5238,...,225377,5006822,1020722.0,2.0,159796.0,65581.0,2815999.0,2190823.0,ITH,ITH4
5,2022-01-20T17:00:00,7,Liguria,725,41,766,34571,35337,2676,6882,...,245343,4026773,1133756.0,2.0,186596.0,58747.0,2129424.0,1897349.0,ITC,ITC3
6,2022-01-20T17:00:00,8,Emilia-Romagna,2465,152,2617,345823,348440,7493,20140,...,875323,12865778,2520924.0,20.0,692311.0,183012.0,7847931.0,5017847.0,ITH,ITH5
7,2022-01-20T17:00:00,9,Toscana,1330,124,1454,178537,179991,3632,13720,...,628124,10400821,4002107.0,8.0,459984.0,168140.0,5860524.0,4540297.0,ITI,ITI1
8,2022-01-20T17:00:00,10,Umbria,202,11,213,25736,25949,-59,2133,...,135626,3254480,604266.0,1.0,100478.0,35148.0,1427572.0,1826908.0,ITI,ITI2
9,2022-01-20T17:00:00,11,Marche,292,50,342,17903,18245,-7,6905,...,204837,2313937,1418084.0,3.0,175473.0,29364.0,1775509.0,538428.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-01-21T17:00:00,1,Piemonte,2053,142,2195,170290,172485,114,14675,...,13045716,3432235.0,9.0,434248.0,333055.0,4407438.0,8638278.0,ITC,ITC1,4.341375e+06
1,2022-01-21T17:00:00,2,Valle d'Aosta,76,6,82,5903,5985,-76,444,...,412979,121396.0,0.0,14502.0,11651.0,130984.0,281995.0,ITC,ITC2,1.255010e+05
2,2022-01-21T17:00:00,3,Lombardia,3503,276,3779,503173,506952,-11855,32677,...,28850371,7404100.0,17.0,1203896.0,722340.0,14081100.0,14769271.0,ITC,ITC4,1.010397e+07
3,2022-01-21T17:00:00,5,Veneto,1516,181,1697,277560,279257,5362,19117,...,22147161,4353395.0,21.0,666702.0,308511.0,8805864.0,13341297.0,ITH,ITH3,4.907704e+06
4,2022-01-21T17:00:00,6,Friuli Venezia Giulia,426,40,466,57762,58228,1926,4712,...,5036074,1023818.0,4.0,161119.0,68959.0,2827093.0,2208981.0,ITH,ITH4,1.211357e+06
5,2022-01-21T17:00:00,7,Liguria,740,42,782,35708,36490,1153,6197,...,4058742,1139236.0,4.0,188170.0,63370.0,2134900.0,1923842.0,ITC,ITC3,1.543127e+06
6,2022-01-21T17:00:00,8,Emilia-Romagna,2459,145,2604,345543,348147,-293,20654,...,12945202,2527253.0,13.0,703260.0,192701.0,7878822.0,5066380.0,ITH,ITH5,4.467118e+06
7,2022-01-21T17:00:00,9,Toscana,1340,125,1465,178923,180388,397,13029,...,10473670,4019375.0,10.0,465775.0,175378.0,5882398.0,4591272.0,ITI,ITI1,3.722729e+06
8,2022-01-21T17:00:00,10,Umbria,196,12,208,25391,25599,-350,2211,...,3270633,606568.0,1.0,101337.0,36500.0,1431446.0,1839187.0,ITI,ITI2,8.802850e+05
9,2022-01-21T17:00:00,11,Marche,298,51,349,17734,18083,-162,5853,...,2332968,1433490.0,3.0,176694.0,33996.0,1781318.0,551650.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-01-21T17:00:00,1,Piemonte,2053,142,2195,170290,172485,114,14675,...,8638278.0,ITC,ITC1,4.341375e+06,Piemonte,2053,6794,142,628,99
1,2022-01-21T17:00:00,2,Valle d'Aosta,76,6,82,5903,5985,-76,444,...,281995.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,76,143,6,33,0
2,2022-01-21T17:00:00,3,Lombardia,3503,276,3779,503173,506952,-11855,32677,...,14769271.0,ITC,ITC4,1.010397e+07,Lombardia,3503,10457,276,1810,0
3,2022-01-21T17:00:00,5,Veneto,1516,181,1697,277560,279257,5362,19117,...,13341297.0,ITH,ITH3,4.907704e+06,Veneto,1516,6000,181,1000,0
4,2022-01-21T17:00:00,6,Friuli Venezia Giulia,426,40,466,57762,58228,1926,4712,...,2208981.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,426,1277,40,175,0
5,2022-01-21T17:00:00,7,Liguria,740,42,782,35708,36490,1153,6197,...,1923842.0,ITC,ITC3,1.543127e+06,Liguria,740,1824,42,221,12
6,2022-01-21T17:00:00,8,Emilia-Romagna,2459,145,2604,345543,348147,-293,20654,...,5066380.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,2459,9001,145,889,0
7,2022-01-21T17:00:00,9,Toscana,1340,125,1465,178923,180388,397,13029,...,4591272.0,ITI,ITI1,3.722729e+06,Toscana,1340,5033,125,570,8
8,2022-01-21T17:00:00,10,Umbria,196,12,208,25391,25599,-350,2211,...,1839187.0,ITI,ITI2,8.802850e+05,Umbria,196,662,12,86,41
9,2022-01-21T17:00:00,11,Marche,298,51,349,17734,18083,-162,5853,...,551650.0,ITI,ITI3,1.518400e+06,Marche,298,1012,51,256,49


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-01-21T17:00:00,3,Lombardia,3503,276,3779,503173,506952,-11855,32677,...,15.25,-11644,-999,202298,26055.0,121,44411,16.15,0.32341,19.06
1,2022-01-21T17:00:00,8,Emilia-Romagna,2459,145,2604,345543,348147,-293,20654,...,16.31,-280,514,79424,6329.0,29,20902,26.00,0.46236,20.06
2,2022-01-21T17:00:00,5,Veneto,1516,181,1697,277560,279257,5362,19117,...,18.10,5392,-2716,142976,10278.0,26,13729,13.37,0.38953,19.87
3,2022-01-21T17:00:00,12,Lazio,1946,207,2153,254747,256900,10127,15314,...,21.95,10091,1891,100878,6890.0,26,5161,15.18,0.26108,12.93
4,2022-01-21T17:00:00,1,Piemonte,2053,142,2195,170290,172485,114,14675,...,22.61,118,-653,102531,17115.0,25,14536,14.31,0.33803,17.67
5,2022-01-21T17:00:00,15,Campania,1339,100,1439,242567,244006,-2429,13682,...,12.72,-2438,-3295,90856,15118.0,21,16090,15.06,0.23647,15.91
6,2022-01-21T17:00:00,9,Toscana,1340,125,1465,178923,180388,397,13029,...,21.93,386,-691,72849,17268.0,34,12598,17.88,0.34999,17.22
7,2022-01-21T17:00:00,16,Puglia,712,67,779,132439,133218,-265,8423,...,13.43,-290,90,72066,10218.0,12,15204,11.69,0.21014,12.99
8,2022-01-21T17:00:00,19,Sicilia,1417,170,1587,200852,202439,5422,7418,...,20.09,5408,-579,46999,46441.0,26,1970,15.78,0.14930,11.25
9,2022-01-21T17:00:00,7,Liguria,740,42,782,35708,36490,1153,6197,...,19.00,1137,-685,31969,5480.0,2,5042,19.38,0.40159,16.30


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,23,67,90,25.56
1,Friuli Venezia Giulia,40,135,175,22.86
2,Piemonte,142,486,628,22.61
3,Lazio,207,736,943,21.95
4,Toscana,125,445,570,21.93
5,Sicilia,170,676,846,20.09
6,Marche,51,205,256,19.92
7,Abruzzo,35,146,181,19.34
8,Liguria,42,179,221,19.00
9,Valle d'Aosta,6,27,33,18.18


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))